In [ ]:
'''
In this notebook, 
A. Using MFCC, ZCR, MELC features Numeric values
B. Applying different data pre-processing techniques such as Scalar, PCA to normalize the data
C. building Model based on SVM/LR algorithm

'''

In [ ]:
import pandas as pd
import os
import glob
import librosa
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import pickle
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [192]:
data1 = pd.read_csv('D:/Users/618757/Downloads/combined.csv')
data2 = pd.read_csv('D:/Users/618757/Downloads/featuresdf_noise_extnd2.csv')
data = pd.concat([data1, data2], axis=0)
data.head()

,Unnamed: 0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,...,contrast6,contrast7,tonnetz1,tonnetz2,tonnetz3,tonnetz4,tonnetz5,tonnetz6,zcr,class_label
0,0.0,-219.254015,49.025944,38.134128,8.716095,13.166282,-3.738123,0.618236,4.393308,0.786051,...,14.893092,27.697563,0.006474,0.011815,0.014826,-0.001648,0.005790,0.009444,0.127813,noise
1,0.0,-165.219021,91.554787,7.419577,17.275317,0.139784,-3.630262,-4.722598,-7.591958,-11.454562,...,16.907168,29.596972,-0.009164,-0.001519,-0.026914,-0.014540,0.008354,-0.003357,0.123505,noise
2,0.0,-266.848935,118.554100,8.920712,55.932076,24.589892,14.573840,14.370409,9.566865,0.335002,...,21.343183,33.245227,-0.011038,0.041138,0.027556,0.026156,0.004832,0.016414,0.038331,noise
3,0.0,-183.045824,103.228989,27.845830,26.464525,8.779398,4.909657,-6.033413,-8.526628,-3.072327,...,17.223171,29.966447,-0.006903,0.032071,0.004660,0.032781,0.002552,0.010910,0.090423,noise
4,0.0,-308.744411,38.235070,30.247514,40.287283,22.950775,-1.223836,7.388353,2.777400,-3.882176,...,18.092151,28.760814,-0.001642,0.016653,-0.009306,0.022628,0.006374,0.003788,0.131564,noise


In [ ]:
# It is to plot the scatter graph between Input feature and output value
for columns in data.columns.values:
    if 'zcr' in columns:
        print('For :', columns)
        plt.scatter(data[columns].values, data['class_label'].values)
        plt.show()

In [193]:
mfcc = data.iloc[:,1:41]
chroma = data.iloc[:,41:53]
mel_spect = data.iloc[:,53:181]
contrast = data.iloc[:,181:188]
tonnetz = data.iloc[:,188:194]
zcr = data.iloc[:,194:195]

In [194]:
from sklearn import preprocessing
import pickle 

min_max_scaler_melc = preprocessing.MinMaxScaler()
mel_spect_transformed = pd.DataFrame(min_max_scaler_melc.fit_transform(mel_spect))
with open('AudioClassification_SVC_Scalar_MELSPEC_04242020_v1.pkl', 'wb') as fid:
    pickle.dump(min_max_scaler_melc, fid)
    
min_max_scaler_mfcc = preprocessing.MinMaxScaler()
mfcc_transformed = pd.DataFrame(min_max_scaler_mfcc.fit_transform(mfcc))
with open('AudioClassification_SVC_Scalar_MFCC_04242020_v1.pkl', 'wb') as fid:
    pickle.dump(min_max_scaler_mfcc, fid)

In [195]:
wrkng_df = pd.concat([mfcc_transformed, mel_spect_transformed], axis=1)

In [196]:
#pca = PCA(n_components=13)
pca = PCA(n_components = 0.95)
pca.fit(wrkng_df)
reduced = pca.transform(wrkng_df)

with open('AudioClassification_SVC_PCA_04242020_v1.pkl', 'wb') as fid:
    pickle.dump(pca, fid)
'''
principalComponents = pca.fit_transform(wrkng_df)
principalDf = pd.DataFrame(data = principalComponents)
'''


'\nprincipalComponents = pca.fit_transform(wrkng_df)\nprincipalDf = pd.DataFrame(data = principalComponents)\n'

In [197]:
principalDf = pd.DataFrame(data = reduced)
principalDf.head()

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0.610675,-0.417791,0.090319,-0.375827,0.026659,0.245480,0.174923,0.024746,-0.090092,-0.047767,...,-0.056722,-0.042400,-0.071184,-0.020169,0.006496,-0.005217,-0.015651,-0.060501,0.055120,-0.024366
1,0.211912,-0.159141,-0.167709,-0.009542,-0.071971,-0.009261,-0.181408,0.039614,0.109877,-0.079114,...,-0.030830,0.003779,-0.008902,0.014221,-0.001941,0.014046,0.023563,-0.009268,0.050787,0.012166
2,0.527976,-0.050951,0.001548,0.113652,-0.281367,0.272108,0.001376,0.039162,-0.004405,0.058693,...,-0.056660,0.017528,-0.039623,-0.058082,0.009154,0.075038,0.087481,0.013595,-0.039756,-0.076471
3,0.303482,-0.270907,-0.200480,-0.023363,-0.074022,0.069142,-0.116553,0.070379,0.011205,-0.042884,...,-0.011621,-0.034204,-0.012747,-0.041267,-0.003620,-0.026628,0.009600,0.035175,0.030529,0.047352
4,0.482836,-0.191745,-0.153059,0.061274,-0.144063,0.149212,-0.152867,-0.023933,0.076110,0.012252,...,0.024157,0.020004,0.024491,-0.054829,-0.071424,-0.002304,0.025598,0.004603,-0.004569,-0.014401


In [199]:
X_train, X_test, y_train, y_test = train_test_split(principalDf, data.class_label, test_size=0.2,random_state=42)

In [200]:
clf = SVC(kernel = 'rbf', gamma=0.7, C = 1)
clf.fit(X_train,y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.7, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [201]:
with open('AudioClassification_SVC_Model_04242020_v1.pkl', 'wb') as fid:
    pickle.dump(clf, fid)

In [202]:
prediction = clf.predict(X_test)
print(prediction)

['speech' 'noise' 'noise' ... 'speech' 'noise' 'noise']


In [203]:
prediction = clf.score(X_test, y_test)
print(prediction)

0.9998701635938717


In [204]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver="lbfgs", multi_class="auto")
model.fit(X_train,y_train)
preds = model.predict(X_test)
print(confusion_matrix(y_test, preds))

with open('AudioClassification_LogisticRegression_Model_04242020_v1.pkl', 'wb') as fid:
    pickle.dump(model, fid)

[[4149    1]
 [   0 3552]]


In [205]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, confusion_matrix

featuresdf_test = pd.DataFrame()
featuresdf_test['label'] = y_test
featuresdf_test['pred_label'] =clf.predict(X_test)
accuracy = accuracy_score(featuresdf_test['label'], featuresdf_test['pred_label'])
precision = precision_score(featuresdf_test['label'], featuresdf_test['pred_label'],average='weighted')
print (accuracy)
print (precision)
confusion_matrix(featuresdf_test['pred_label'], featuresdf_test['label'])


0.9998701635938717
0.9998702001366261


array([[4149,    0],
       [   1, 3552]], dtype=int64)

In [234]:
import pandas as pd
import os
import glob
import librosa
import numpy as np

def validateAgainstSpearData(min_max_scaler_mfcc, min_max_scaler_melc, pca ):
    parent_dir = '/SpEAR-speech-database-master/data'
    sub_dirs= ['Lombard', 'Monaural', 'Noisy_Recordings', 'TIMIT']
    sub_dirs= ['noise', 'good']
    file_ext = "*.wav"
    
    features_df = pd.DataFrame()
    lables = []
    file_names = []

    for l, sub_dir in enumerate(sub_dirs):
        print('Processing folder: ', sub_dir)
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):

            audio, sample_rate = librosa.load(fn, sr=22050, res_type='kaiser_fast')
            # feature = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
            file_names.append(sub_dir+'/'+fn)

            mfcc_df = pd.DataFrame([np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)])
            mfcc_transformed = pd.DataFrame(min_max_scaler_mfcc.transform(mfcc_df))

            mel_df = pd.DataFrame([np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)])
            mel_spect_transformed = pd.DataFrame(min_max_scaler_melc.transform(mel_df))

            zcr = pd.DataFrame([np.mean(librosa.feature.zero_crossing_rate(y= audio))])

            wrkng_df1 = pd.concat([mfcc_transformed, mel_spect_transformed], axis=1)

            features_df = pd.concat([features_df, wrkng_df1], ignore_index=True)

            if sub_dir == 'noise':
                lables.append('noise')
                
            if sub_dir == 'good':
                lables.append('speech')
                
            if sub_dir == 'Lombard':
                lables.append('noise')

            if sub_dir == 'Monaural':
                lables.append('speech')

            if sub_dir == 'Noisy_Recordings':
                lables.append('noise')

            if sub_dir == 'TIMIT':
                lables.append('speech')

    reduced_test_df = pca.transform(features_df)
    
    return reduced_test_df, lables
#featuresdf = pd.DataFrame(columns=['feature','class_label'])

In [221]:
def validateAgainstNoiseData(min_max_scaler_mfcc, min_max_scaler_melc, pca ):
    features_df = pd.DataFrame()

    parent_dir = '/noise'
    file_ext = "*.wav"

    file_names = []
    lables = []
    n=0 
    for fn in glob.glob(os.path.join(parent_dir, file_ext)):
        audio, sample_rate = librosa.load(fn, sr=22050, res_type='kaiser_fast')
        # feature = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        file_names.append(sub_dir+'/'+fn)

        mfcc_df = pd.DataFrame([np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)])
        mfcc_transformed = pd.DataFrame(min_max_scaler_mfcc.transform(mfcc_df))

        mel_df = pd.DataFrame([np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)])
        mel_spect_transformed = pd.DataFrame(min_max_scaler_melc.transform(mel_df))

        zcr = pd.DataFrame([np.mean(librosa.feature.zero_crossing_rate(y= audio))])

        wrkng_df1 = pd.concat([mfcc_transformed, mel_spect_transformed], axis=1)
        
        features_df = pd.concat([features_df, wrkng_df1], ignore_index=True)
        lables.append('noise')
    
    reduced_test_df = pca.transform(features_df)
    return reduced_test_df, lables

#featuresdf = pd.DataFrame(columns=['feature','class_label'])

In [238]:
import pandas as pd
import os
import glob
import librosa
import numpy as np

def validateAgainstMixData(min_max_scaler_mfcc, min_max_scaler_melc, pca ):
    parent_dir = 'D:/Abhishek/Machine Learning Models/Audio Data Analysis/Models/audio_data'
    #sub_dirs= ['Lombard', 'Monaural', 'Noisy_Recordings', 'TIMIT']
    sub_dirs= ['noise', 'good']
    sub_sub_good_dirs= ['flicker', 'spear_good', 'urban']
    sub_sub_noise_dirs= ['audio_mixed_noise', 'spear_noise', 'only_noise']
    file_ext = "*.wav"
    
    features_df = pd.DataFrame()
    lables = []
    file_names = []

    for l, sub_dir in enumerate(sub_dirs):
        print('Processing folder: ', sub_dir)
        if sub_dir == 'good':
            processed_sub_dir = sub_sub_good_dirs
        
        if sub_dir == 'noise':
            processed_sub_dir = sub_sub_noise_dirs   
            
        for l, sub_sub_dir in enumerate(processed_sub_dir):
            
            for fn in glob.glob(os.path.join(parent_dir, sub_dir, sub_sub_dir, file_ext)):

                audio, sample_rate = librosa.load(fn, sr=22050, res_type='kaiser_fast')
                # feature = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
                file_names.append(sub_dir+'/'+fn)

                mfcc_df = pd.DataFrame([np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)])
                mfcc_transformed = pd.DataFrame(min_max_scaler_mfcc.transform(mfcc_df))

                mel_df = pd.DataFrame([np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)])
                mel_spect_transformed = pd.DataFrame(min_max_scaler_melc.transform(mel_df))

                zcr = pd.DataFrame([np.mean(librosa.feature.zero_crossing_rate(y= audio))])

                wrkng_df1 = pd.concat([mfcc_transformed, mel_spect_transformed], axis=1)

                features_df = pd.concat([features_df, wrkng_df1], ignore_index=True)

                if sub_dir == 'noise':
                    lables.append('noise')

                if sub_dir == 'good':
                    lables.append('speech')

    reduced_test_df = pca.transform(features_df)
    
    return reduced_test_df, lables
#featuresdf = pd.DataFrame(columns=['feature','class_label'])

In [235]:
def modelEvaluation(reduced_test_df, lables, model1) :
    
    featuresdf_test = pd.DataFrame()
    featuresdf_test['label'] = lables
    featuresdf_test['pred_label'] =model1.predict(reduced_test_df)
    
    accuracy = accuracy_score(featuresdf_test['label'], featuresdf_test['pred_label'])
    precision = precision_score(featuresdf_test['label'], featuresdf_test['pred_label'],average='weighted')
    
    print("Accuracy Score: ", accuracy_score(featuresdf_test['label'], featuresdf_test['pred_label']))
    print("F1 Score: ", f1_score(featuresdf_test['label'], featuresdf_test['pred_label'], average='weighted'))
    print("Precision Score: ", precision_score(featuresdf_test['label'], featuresdf_test['pred_label'], average='weighted'))
    print("Recall Score: ", recall_score(featuresdf_test['label'], featuresdf_test['pred_label'], average='weighted'))
    print("Classification report: \n", metrics.classification_report(featuresdf_test['label'], featuresdf_test['pred_label']))
    print(confusion_matrix(featuresdf_test['pred_label'], featuresdf_test['label']))

In [239]:
# CLF Model Against Spear data
reduced_test_df, lables = validateAgainstMixData(min_max_scaler_mfcc, min_max_scaler_melc, pca )
modelEvaluation(reduced_test_df, lables, clf)

Processing folder:  noise
Processing folder:  good
Before performing PCA, col lenght:  168
After performing PCA, col lenght:  48
Accuracy Score:  0.4888888888888889
F1 Score:  0.4751984126984127
Precision Score:  0.654040404040404
Recall Score:  0.4888888888888889
Classification report: 
               precision    recall  f1-score   support

       noise       0.79      0.32      0.45        60
      speech       0.38      0.83      0.52        30

    accuracy                           0.49        90
   macro avg       0.59      0.57      0.49        90
weighted avg       0.65      0.49      0.48        90

[[19  5]
 [41 25]]


In [240]:
# LR Model Against Spear data
reduced_test_df, lables = validateAgainstMixData(min_max_scaler_mfcc, min_max_scaler_melc, pca )
modelEvaluation(reduced_test_df, lables, model)

Processing folder:  noise
Processing folder:  good
Before performing PCA, col lenght:  168
After performing PCA, col lenght:  48
Accuracy Score:  0.6
F1 Score:  0.6071428571428571
Precision Score:  0.7037037037037036
Recall Score:  0.6
Classification report: 
               precision    recall  f1-score   support

       noise       0.83      0.50      0.62        60
      speech       0.44      0.80      0.57        30

    accuracy                           0.60        90
   macro avg       0.64      0.65      0.60        90
weighted avg       0.70      0.60      0.61        90

[[30  6]
 [30 24]]


In [224]:
# CLF Model Against Noise data
reduced_test_df, lables = validateAgainstNoiseData(min_max_scaler_mfcc, min_max_scaler_melc, pca )
modelEvaluation(reduced_test_df, lables, clf)

Before performing PCA, col  168
After performing PCA, col lenght:  48
Accuracy Score:  0.4444444444444444
F1 Score:  0.6153846153846153
Precision Score:  1.0
Recall Score:  0.4444444444444444
Classification report: 
               precision    recall  f1-score   support

       noise       1.00      0.44      0.62        18
      speech       0.00      0.00      0.00         0

    accuracy                           0.44        18
   macro avg       0.50      0.22      0.31        18
weighted avg       1.00      0.44      0.62        18

[[ 8  0]
 [10  0]]


D:\Users\618757\Anaconda3\envs\tensorFlowEnv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\618757\Anaconda3\envs\tensorFlowEnv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [225]:
# LR Model Against Noise data
reduced_test_df, lables = validateAgainstNoiseData(min_max_scaler_mfcc, min_max_scaler_melc, pca )
modelEvaluation(reduced_test_df, lables, model)

Before performing PCA, col  168
After performing PCA, col lenght:  48
Accuracy Score:  0.6111111111111112
F1 Score:  0.7586206896551725
Precision Score:  1.0
Recall Score:  0.6111111111111112
Classification report: 
               precision    recall  f1-score   support

       noise       1.00      0.61      0.76        18
      speech       0.00      0.00      0.00         0

    accuracy                           0.61        18
   macro avg       0.50      0.31      0.38        18
weighted avg       1.00      0.61      0.76        18

[[11  0]
 [ 7  0]]


D:\Users\618757\Anaconda3\envs\tensorFlowEnv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\618757\Anaconda3\envs\tensorFlowEnv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
